# Premature babies | gestational age & Cooling therapy treatment

In [1]:
headers = {
  'Accept': 'application/fhir+json; fhirVersion=4.0',
  'Content-Type': 'application/fhir+json; fhirVersion=4.0',
  'Cookie': 'ARRAffinity=a6e48b9e9d2653435be7b61998d8624b44115214104213d6c8b8c526cc56dc70; ARRAffinitySameSite=a6e48b9e9d2653435be7b61998d8624b44115214104213d6c8b8c526cc56dc70'
}

In [2]:
import uuid
import pandas as pd
import requests
import json
import datetime
import random
import pytz
import pandas as pd


In [3]:
df1 = pd.DataFrame(columns=['year', 'gestational age'])
df2 = pd.DataFrame(columns=['year'])

def generate_random_datetime(year):
    start_date = datetime.datetime(year, 1, 1)
    end_date = datetime.datetime(year, 12, 31)

    # Calculate the total number of seconds within the specified year
    total_seconds = (end_date - start_date).total_seconds()

    # Generate a random duration within the specified year
    random_duration = random.randint(0, int(total_seconds))

    # Add the random duration to the start date
    random_datetime = start_date + datetime.timedelta(seconds=random_duration)

    # Convert the random datetime to the desired format
    formatted_datetime = random_datetime.strftime("%Y-%m-%dT%H:%M:%S")
    formatted_datetime = str(formatted_datetime)+'+02:00'

    return formatted_datetime

def generate_random_gestational_age():
    # Generate a random gestational age with less prematurity during COVID times
    if random.random() < 0.7:  # 70% chance of being full term
        # source: https://pathways.org/adjusting-age-for-prematurity/
        gestational_age = random.randint(38, 42)  # Full-term range (38-42 weeks)
    else:
        gestational_age = random.randint(28, 37)  # Preterm range (28-37 weeks)

    return gestational_age

def generate_dfs(datetime, gestational_age):
    # Placeholder function to generate an observation
    # Replace this function with your actual implementation
    # You can access the datetime and gestational_age variables here
    # to generate the observation data for each patient
    #print(f"Observation generated for datetime: {datetime} and gestational age: {gestational_age}")
    
    df1.loc[len(df1)] = {'year': str(datetime)[:4], 'gestational age': gestational_age}


In [4]:
def generate_patients():
    years = range(1960,1980)
    for num_pat in range(0,20):
        birth = generate_random_datetime(years[num_pat])
        data = {
            "resourceType": "Patient",
            "id": f"patient-{str(num_pat)}",
            "active": True,
            "name": [
                {
                    "use": "official",
                    "family": "Patient dumb",
                    "given": [
                        "Dumb","Patient"
                    ]
                },
                {
                    "use": "usual",
                    "given": [
                        "Dumb pat"
                    ]
                }
            ],
            "gender": "female",
            "birthDate": str(birth).split('T')[0]
        }
        url = f"https://firelyasserver-backup.azurewebsites.net/Patient/patient-{str(num_pat)}"
        payload = json.dumps(data)
        response = requests.request("PUT", url, data=payload, headers=headers)
        if response.status_code != 201 and response.status_code != 200:
            print(f'Female patient not created {num_pat}')
            print(payload)

    for num_pat_male in range(0,20):
        birth = generate_random_datetime(years[num_pat_male])
        data = {
            "resourceType": "Patient",
            "id": f"patient-male-{str(num_pat_male)}",
            "active": True,
            "name": [
                {
                    "use": "official",
                    "family": "Patient dumb",
                    "given": [
                        "Dumb","Patient"
                    ]
                },
                {
                    "use": "usual",
                    "given": [
                        "Dumb pat"
                    ]
                }
            ],
            "gender": "male",
            "birthDate": str(birth).split('T')[0]
        }
        url = f"https://firelyasserver-backup.azurewebsites.net/Patient/patient-male-{str(num_pat_male)}"
        payload = json.dumps(data)
        response = requests.request("PUT", url, data=payload, headers=headers)
        if response.status_code != 201 and response.status_code != 200:
            print(f'Male patient not created {num_pat_male}')


In [5]:

def generate_observation(datetime, value):
    ID = str(uuid.uuid4())
    random_patient = random.randint(0, 20)
    observation = {
        "resourceType": "Observation",
        "id": ID,
        "status": "final",
        "category": [
            {
                "coding": [
                    {
                        "system": "gestational-age",
                        "code": "gestational-age",
                        "display": "Gestational age"
                    }
                ]
            }
        ],
        "code": {
            "coding": [
                {
                    "system": "gestAge",
                    "code": "gestAge",
                    "display": "Gestational Age"
                }
            ],
            "text": "Gestational Age"
        },
        "subject": {
            "reference": f"https://firelyasserver-backup.azurewebsites.net/Patient/patient-{random_patient}"
        },
        "encounter": {
            "reference": "no-ref"
        },
        "effectiveDateTime": datetime #"2020-08-31T22:13:22+02:00",
        ,
        "issued": "2020-08-31T22:13:22.922+02:00",
        "valueQuantity": {
            "value": value,
            "unit": "weeks"
        }
    }
    url = f"https://firelyasserver-backup.azurewebsites.net/Observation/{ID}"
    payload = json.dumps(observation)
    response = requests.request("PUT", url, data=payload, headers=headers)
    if response.status_code != 201 and response.status_code != 200:
        print('Observation ot created')


In [6]:
def generate_procedure_coolingtherapy(datetime):
    ID = str(uuid.uuid4())
    random_patient = random.randint(0, 20)
    procedure = {
                "resourceType": "Procedure",
                "id": ID,
                "status": "completed",
                "code": {
                    "coding": [
                        {
                            "system": "CT",
                            "code": "CT",
                            "display": "Cooling therapy"
                        }
                    ],
                    "text": "Cooling therapy"
                },
                "subject": {
                    "reference": f"https://firelyasserver-backup.azurewebsites.net/Patient/patient-{random_patient}"
                },
                "encounter": {
                    "reference": "no-ref"
                },
                "performedDateTime": datetime,
                "location": {
                    "reference": "AMC",
                    "display": "AMC"
                }
            }
    url = f"https://firelyasserver-backup.azurewebsites.net/Procedure/{ID}"
    payload = json.dumps(procedure)
    response = requests.request("PUT", url, data=payload, headers=headers)
    if response.status_code != 201 and response.status_code != 200:
        print('Procedure not created')


In [7]:
generate_patients()


In [9]:

for year in range(2017, 2024):
    for _ in range(300):
        random_datetime = generate_random_datetime(year)
        random_gestational_age = generate_random_gestational_age()

        if year == 2020 or year == 2021 or year == 2022:  # Considering the COVID period
            random_gestational_age += random.randint(2, 4)  # Increase the gestational age by 2 to 4 weeks
            
            if random.random() < 0.80:  # 80% chance of cooling therapy during covid
                generate_procedure_coolingtherapy(random_datetime)
                df2.loc[len(df2)] = {'year': str(random_datetime)[:4]}

        
        generate_dfs(random_datetime, random_gestational_age)
        generate_observation(random_datetime, random_gestational_age)
        
        if year == 2017 or year == 2018 or year == 2019 or year == 2023:
            if random.random() < 0.60:  # 60% chance of cooling therapy during covid
                generate_procedure_coolingtherapy(random_datetime)
                df2.loc[len(df2)] = {'year': str(random_datetime)[:4]}
        
